In [1]:
# =========================
# ProjectTwoDashboard.ipynb
# Single-cell Dash app
# CS-499 Milestone 4 (Databases)
# Local MongoDB version
# =========================

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import base64
import pandas as pd
import numpy as np
import os

# Import your enhanced CRUD module
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Connect to LOCAL MongoDB with default params (localhost:27017, no auth)
db = AnimalShelter()

# Utility: safe DataFrame from list of dicts, drop _id if present
def df_from_records(records):
    df = pd.DataFrame.from_records(records) if records else pd.DataFrame()
    if "_id" in df.columns:
        df = df.drop(columns=["_id"])
    return df

# Optional: seed from CSV if your local DB is empty
# (CSV must be in the same folder as this notebook)
csv_path = "aac_shelter_outcomes.csv"
if db.collection.estimated_document_count() == 0 and os.path.exists(csv_path):
    try:
        seed_df = pd.read_csv(csv_path)
        # Insert in batches to avoid memory issues
        batch_size = 5000
        records = seed_df.to_dict(orient="records")
        for i in range(0, len(records), batch_size):
            db.collection.insert_many(records[i:i+batch_size], ordered=False)
        print(f"Seeded {len(records)} records from {csv_path}")
    except Exception as e:
        print(f"CSV seed skipped due to error: {e}")

# Initial load (unfiltered)
df = df_from_records(db.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Grazioso Salvare Logo
image_filename = 'Grazioso Salvare Logo.png'
try:
    with open(image_filename, 'rb') as f:
        encoded_image = base64.b64encode(f.read()).decode()
except FileNotFoundError:
    print("Logo file not found.")
    encoded_image = ""

# Rescue breed presets (used in SNHU CS-340 examples)
RESCUE_BREEDS = {
    "Water Rescue": ["Labrador Retriever", "Chesapeake Bay Retriever", "Newfoundland"],
    "Mountain or Wilderness Rescue": ["German Shepherd", "Alaskan Malamute", "Border Collie"],
    "Disaster or Individual Tracking": ["Bloodhound", "Rottweiler", "Doberman Pinscher"]
}

# Dropdown options (robust if columns are missing)
OUTCOME_OPTIONS = [{"label": x, "value": x} for x in sorted(df["outcome_type"].dropna().unique())] if "outcome_type" in df.columns else []
SEX_OPTIONS = [{"label": x, "value": x} for x in sorted(df["sex_upon_outcome"].dropna().unique())] if "sex_upon_outcome" in df.columns else []

app.layout = html.Div([
    html.Center([
        html.Img(src=f'data:image/png;base64,{encoded_image}', style={'height':'100px'}) if encoded_image else html.Div(),
        html.B(html.H1('CS-340 Dashboard — Animal Shelter (Local MongoDB)')),
        html.P("Developed by Niaz Khan", style={'fontSize': '18px'})
    ]),
    html.Hr(),

    # ----- Controls -----
    html.Div([
        html.Div([
            html.Label("Rescue Type"),
            dcc.RadioItems(
                id='filter-type',
                options=[
                    {'label': 'Water Rescue', 'value': 'Water Rescue'},
                    {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                    {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                    {'label': 'Reset', 'value': 'Reset'}
                ],
                value='Reset',
                labelStyle={'display': 'inline-block', 'margin-right': '20px'}
            ),
        ], style={"marginBottom": "10px"}),

        html.Div([
            html.Label("Outcome Type"),
            dcc.Dropdown(
                id="ddl-outcome",
                options=[{"label": "Any", "value": ""}] + OUTCOME_OPTIONS,
                value=""
            ),
        ], style={"width": "30%", "display": "inline-block", "paddingRight": "10px"}),

        html.Div([
            html.Label("Sex Upon Outcome"),
            dcc.Dropdown(
                id="ddl-sex",
                options=[{"label": "Any", "value": ""}] + SEX_OPTIONS,
                value=""
            ),
        ], style={"width": "30%", "display": "inline-block", "paddingRight": "10px"}),

        html.Div([
            html.Label("Breed contains (partial text)"),
            dcc.Input(id="txt-breed", type="text", placeholder="e.g., mix", debounce=True, style={"width": "100%"}),
        ], style={"width": "30%", "display": "inline-block"}),

        html.Div(style={"height": "10px"}),

        html.Div([
            html.Label("Age upon outcome (weeks)"),
            dcc.RangeSlider(
                id='rng-age',
                min=0, max=520, step=1,
                value=[0, 104],  # default: up to 2 years
                tooltip={"placement": "bottom", "always_visible": False}
            ),
        ], style={"padding": "10px 0 0 0"}),

    ], style={"padding": "0 10px"}),

    html.Hr(),

    # ----- Data Table -----
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        row_selectable='single',
        selected_rows=[],
        style_table={'overflowX': 'auto'},
        style_cell={
            'minWidth': '120px', 'width': '150px', 'maxWidth': '220px',
            'whiteSpace': 'normal'
        },
        style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'}
    ),

    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ]),

    html.Hr(),
    html.Pre(id='debug-output', style={'fontSize': '12px', 'color': '#555'})
])

#############################################
# Interaction Between Components / Controller
#############################################

# Update the table with advanced filters + rescue presets
@app.callback(
    Output('datatable-id', 'data'),
    Output('datatable-id', 'columns'),
    Input('filter-type', 'value'),
    Input('ddl-outcome', 'value'),
    Input('ddl-sex', 'value'),
    Input('rng-age', 'value'),
    Input('txt-breed', 'value')
)
def update_dashboard(filter_type, outcome_type, sex, age_range, breed_text):
    # 1) Pull via advanced query
    outcome = outcome_type if outcome_type else None
    sex_val = sex if sex else None
    breed_contains = breed_text.strip() if breed_text else None

    try:
        records = db.read_advanced(
            animal_type=None,
            outcome_type=outcome,
            min_age_wk=age_range[0] if age_range else None,
            max_age_wk=age_range[1] if age_range else None,
            breed_contains=breed_contains,
            sex=sex_val,
            limit=2000
        )
    except Exception as e:
        # In case of any query errors, return empty
        print(f"read_advanced error: {e}")
        records = []

    dff = df_from_records(records)

    # 2) Apply rescue preset (breed inclusion + <=104 weeks) if selected
    if filter_type and filter_type in RESCUE_BREEDS and "breed" in dff.columns:
        dff = dff[dff["breed"].isin(RESCUE_BREEDS[filter_type])]
        if "age_upon_outcome_in_weeks" in dff.columns:
            dff = dff[dff["age_upon_outcome_in_weeks"] <= 104]

    # 3) Ensure numeric coords and drop rows without coords for map stability
    for col in ["location_lat", "location_long"]:
        if col in dff.columns:
            dff[col] = pd.to_numeric(dff[col], errors="coerce")

    essential_fields = [c for c in ["name", "breed", "location_lat", "location_long"] if c in dff.columns]
    if essential_fields:
        dff = dff.dropna(subset=essential_fields)

    # Update column model dynamically in case fields vary
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in dff.columns]

    return dff.to_dict('records'), columns


# Aggregation -> Bar chart (Counts by Animal Type, optionally filtered by outcome)
@app.callback(
    Output('graph-id', "children"),
    Input('ddl-outcome', 'value')
)
def update_graphs(outcome_type):
    try:
        match = {"outcome_type": outcome_type} if outcome_type else {}
        stats = db.aggregate_adoption_stats(group_by="animal_type", match=match)
        if not stats:
            return [html.P("No aggregation data to display.")]
        adf = pd.DataFrame(stats)
        adf.rename(columns={"_id": "animal_type", "count": "count"}, inplace=True)
        title = "Counts by Animal Type" if not outcome_type else f"Counts by Animal Type (Outcome: {outcome_type})"
        fig = plotly_bar = px.bar(adf, x="animal_type", y="count", title=title)
        return [dcc.Graph(figure=fig)]
    except Exception as e:
        print(f"aggregate_adoption_stats error: {e}")
        return [html.P("No data to display.")]


# Highlight selected columns visually
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    Input('datatable-id', 'selected_columns')
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# Update the map from selected row (fallback to first row)
@app.callback(
    Output('map-id', "children"),
    Input('datatable-id', "derived_virtual_data"),
    Input('datatable-id', "derived_virtual_selected_rows")
)
def update_map(viewData, selected_rows):
    if not viewData:
        return [html.P("No map data to display.")]

    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty:
        return [html.P("No map data to display.")]

    # Ensure numeric
    for col in ["location_lat", "location_long"]:
        if col in dff.columns:
            dff[col] = pd.to_numeric(dff[col], errors="coerce")

    if "location_lat" not in dff.columns or "location_long" not in dff.columns:
        return [html.P("Latitude/Longitude data not available.")]

    dff = dff.dropna(subset=["location_lat", "location_long"])
    if dff.empty:
        return [html.P("Latitude/Longitude data not available.")]

    row_idx = (selected_rows[0] if selected_rows else 0)
    row_idx = min(row_idx, len(dff) - 1)

    lat = float(dff.iloc[row_idx]["location_lat"])
    lon = float(dff.iloc[row_idx]["location_long"])
    breed = dff.iloc[row_idx].get('breed', 'Breed Unknown')
    name = dff.iloc[row_idx].get('name', 'Unnamed')

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(name)
                ])
            ])
        ])
    ]


# Debug helper
@app.callback(
    Output('debug-output', 'children'),
    Input('datatable-id', 'data')
)
def debug_output(data):
    if data and len(data) > 0:
        return f"Sample record:\n{data[0]}"
    else:
        return "No data loaded."

# Run the dashboard (use a different port to avoid conflict)
app.run_server(debug=True, port=8051)

/Users/niaz/Desktop/Animal Shelter - Artifact 3/venv/lib/python3.9/site-packages/dash/dash.py:516: UserWarning: JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.
  warnings.warn(


Dash app running on http://127.0.0.1:8051/
